# Bitcoin News Keyword Trend Analysis: End-to-End Example

This notebook demonstrates a complete application of TextBlob for analyzing keyword trends in Bitcoin news and their correlation with Bitcoin price movements.

We'll walk through each step of the workflow:
1. Fetch news and price data
2. Extract keywords using TextBlob
3. Analyze trends and correlations
4. Test for causality using Granger tests
5. Visualize results
6. Generate a dashboard

In [ ]:
# Import our utility module containing all the required functions
import textblob1_utils as utils
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)
os.makedirs('dashboard', exist_ok=True)

# Configure visualizations
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

## Step 1: Fetch News and Price Data

First, we'll fetch Bitcoin-related news and historical price data. For this example, we'll use sample data.

In [ ]:
# Fetch Bitcoin news (using sample data without API key)
news_df = utils.fetch_bitcoin_news(api_key=None, days=30)
print(f"Fetched {len(news_df)} news articles")
news_df.head()

In [ ]:
# Fetch Bitcoin price data (using sample data)
price_df = utils.fetch_bitcoin_prices(days=30, interval='daily')
print(f"Fetched {len(price_df)} days of price data")
price_df.head()

In [ ]:
# Get current Bitcoin price for dashboard
current_price = utils.fetch_current_bitcoin_price()
print(f"Current Bitcoin price: ${current_price['price']:.2f} at {current_price['timestamp']}")

## Step 2: Extract Keywords Using TextBlob

Now we'll use TextBlob to extract important keywords (noun phrases) from the news articles.

In [ ]:
# Extract keywords from news articles
news_with_keywords = utils.extract_keywords(news_df, min_frequency=1, include_titles=True)

# Display example of extracted keywords
print("Sample of extracted keywords:")
for i, row in news_with_keywords.head(3).iterrows():
    print(f"Article: {row['title']}")
    print(f"Keywords: {', '.join(row['keywords'][:5]) if len(row['keywords']) > 0 else 'None'}")
    print("---")

## Step 3: Analyze Trends and Correlations

Next, we'll analyze the trends by counting keyword frequencies, merging with price data, and computing correlations.

In [ ]:
# Analyze trends by merging keyword counts with price data
merged_df = utils.analyze_trends(news_with_keywords, price_df, time_window='daily', min_freq=1)

# Display the top keywords by frequency
print("Top 10 keywords by frequency:")
top_keywords = merged_df.drop_duplicates('keyword').sort_values('frequency', ascending=False).head(10)
top_keywords[['keyword', 'frequency', 'correlation']].reset_index(drop=True)

## Step 4: Perform Granger Causality Tests

Now we'll perform Granger causality tests to determine if keywords can predict price changes or vice versa.

In [ ]:
# Run Granger causality tests
causality_results = utils.run_granger_tests(merged_df, top_n=10, max_lag=3)

# Display causality results
if causality_results is not None and not causality_results.empty:
    print("Granger Causality Test Results:")
    causality_results[['keyword', 'correlation', 'k2p_significant', 'p2k_significant']].sort_values(
        'correlation', ascending=False)

## Step 5: Visualize Results

Let's visualize the results with plots showing keyword frequency vs. price and correlation heatmaps.

In [ ]:
# Plot keyword frequency vs. price for the top correlated keyword
if not merged_df.empty and len(merged_df['keyword'].unique()) > 0:
    top_keyword = top_keywords.iloc[0]['keyword']
    print(f"Plotting frequency vs. price for top keyword: {top_keyword}")
    utils.plot_keyword_vs_price(merged_df, top_keyword, save_fig=True)

In [ ]:
# Create a heatmap of top keywords and their price correlations
utils.plot_keyword_price_heatmap(merged_df, top_n=10, save_fig=True)

## Step 6: Generate Interactive Dashboard

Finally, we'll generate an interactive HTML dashboard summarizing our analysis.

In [ ]:
# Generate interactive dashboard
dashboard_path = utils.generate_dashboard(merged_df, output_path='dashboard/index.html')
print(f"Dashboard generated at: {dashboard_path}")

# Try to open the dashboard in a browser
try:
    import webbrowser
    webbrowser.open(dashboard_path)
except:
    print("Could not automatically open dashboard. Please open the file manually.")

## Conclusion

This notebook has demonstrated the complete workflow for analyzing Bitcoin news keyword trends using TextBlob:

1. **Data Collection**: We fetched Bitcoin news and price data
2. **Keyword Extraction**: We used TextBlob to extract meaningful keywords from articles
3. **Trend Analysis**: We analyzed frequency trends and correlations with price movements
4. **Causality Testing**: We used Granger tests to assess potential causal relationships
5. **Visualization**: We created plots showing keyword trends and correlations
6. **Dashboard**: We generated an interactive HTML dashboard summarizing our findings

The entire process is automated through our utility functions in `textblob1_utils.py`. To use this with real data, simply provide a valid NewsAPI key to the `fetch_bitcoin_news` function.